In [3]:
import os
import re
import json
import requests
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm

# === Settings ===
API_KEY = "YOUR API KEY"
MODEL_ID = "openai/gpt-4o"
SAVE_PATH = "/content/drive/MyDrive/ML/openrouter_gpt4o_pm25.csv"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
}
DELAY_BETWEEN_QUERIES = 2  # seconds

# === Load Data ===
df = pd.read_csv("/content/drive/MyDrive/ML/Ground_Truth_2023_Final.csv")

if "YearMonth" in df.columns:
    df["YearMonth"] = pd.to_datetime(df["YearMonth"], format="%Y-%m")
else:
    raise KeyError("CSV does not contain 'YearMonth' date column. Please check column names.")

# === Prompt Templates ===
SYSTEM_PROMPT = (
    "You are an air pollution assistant. "
    "Strictly respond to queries with a single real number only. "
    "Do not include any units, explanation, or punctuation. Just a single number."
)
USER_TEMPLATE = (
    "What is the average PM2.5 concentration (in μg/m³) in {city}, {state} during {month}, {year}? "
    "Give a single number only.\n"
)

# === Resume logic ===
rows = []
if os.path.exists(SAVE_PATH):
    existing_df = pd.read_csv(SAVE_PATH)
    print(f"Resuming from saved file with {len(existing_df)} rows.")
    processed_keys = set(zip(existing_df["city"], existing_df["state"], existing_df["month"]))
    rows = existing_df.to_dict("records")
else:
    print("No previous file found. Starting fresh.")
    processed_keys = set()

# === Query single item with infinite retry ===
def query_llm_single(city, state, month_name, year):
    url = "https://openrouter.ai/api/v1/chat/completions"

    user_prompt = USER_TEMPLATE.format(
        city=city.title(),
        state=state.title(),
        month=month_name,
        year=year,
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ]

    payload = {
        "model": MODEL_ID,
        "messages": messages,
    }

    while True:  # Infinite retry until success
        try:
            response = requests.post(url, headers=HEADERS, data=json.dumps(payload))
            response.raise_for_status()
            data = response.json()

            text = data["choices"][0]["message"]["content"]
            match = re.search(r"\d+(\.\d+)?", text)
            if match:
                value = float(match.group())
                return value
            else:
                print(f"No numeric result for {city}, {state}, {month_name} {year}. Retrying in 30 seconds...")
        except Exception as e:
            print(f"Error for {city}, {state}, {month_name} {year}: {str(e)}. Retrying in 30 seconds...")

        time.sleep(30)  # Wait 30 seconds before retry

# === Prepare unprocessed items ===
items_to_process = []
for _, row in df.iterrows():
    city = row["city"]
    state = row["state"]
    year = row["YearMonth"].year
    month_name = row["YearMonth"].strftime("%B")
    key = (city, state, month_name)
    if key in processed_keys:
        continue
    items_to_process.append((city, state, month_name, year))

# === Process one by one ===
counter = len(rows)
total_items = len(items_to_process)

for idx, (city, state, month_name, year) in enumerate(tqdm(items_to_process, desc="Querying")):
    print(f"\n[{datetime.now()}] Processing {idx+1}/{total_items}: {city}, {state}, {month_name} {year}")

    pred = query_llm_single(city, state, month_name, year)

    rows.append({
        "city": city,
        "state": state,
        "year": year,
        "month": month_name,
        "model": MODEL_ID,
        "pm2.5": pred,
    })
    counter += 1

    # Save after every query
    pd.DataFrame(rows).to_csv(SAVE_PATH, index=False)
    print(f"[{datetime.now()}] Saved at {counter} rows. PM2.5 value: {pred}")

    # Wait 30 seconds before next query (except for the last one)
    if idx < total_items - 1:
        print(f"Waiting {DELAY_BETWEEN_QUERIES} seconds before next query...")
        time.sleep(DELAY_BETWEEN_QUERIES)

print(f"\nDone! Saved total {len(rows)} rows to {SAVE_PATH}")


Resuming from saved file with 289 rows.


Querying:   0%|          | 0/2123 [00:00<?, ?it/s]


[2025-11-28 12:13:00.472275] Processing 1/2123: Bathinda, Punjab, February 2023
[2025-11-28 12:13:01.264752] Saved at 290 rows. PM2.5 value: 147.0
Waiting 2 seconds before next query...


Querying:   0%|          | 1/2123 [00:02<1:38:46,  2.79s/it]


[2025-11-28 12:13:03.266091] Processing 2/2123: Bathinda, Punjab, March 2023
[2025-11-28 12:13:03.682203] Saved at 291 rows. PM2.5 value: 156.0
Waiting 2 seconds before next query...


...


[2025-11-28 13:54:14.455285] Processing 2122/2123: Yamuna Nagar, Haryana, November 2023
[2025-11-28 13:54:14.889774] Saved at 2411 rows. PM2.5 value: 73.0
Waiting 2 seconds before next query...


Querying: 100%|█████████▉| 2122/2123 [1:41:16<00:02,  2.51s/it]


[2025-11-28 13:54:16.891182] Processing 2123/2123: Yamuna Nagar, Haryana, December 2023


Querying: 100%|██████████| 2123/2123 [1:41:16<00:00,  2.86s/it]

[2025-11-28 13:54:17.382696] Saved at 2412 rows. PM2.5 value: 79.0

Done! Saved total 2412 rows to /content/drive/MyDrive/ML/openrouter_gpt4o_pm25.csv
